# 食物声音识别

In [1]:
import os
import moxing as mox
if not os.path.exists('train_sample'):
    mox.file.copy('obs://qwl/food_voice/data/train_sample.zip', 'train_sample.zip')
    !unzip -qq train_sample.zip
    !\rm train_sample.zip
if not os.path.exists('test_a'):
    mox.file.copy('obs://qwl/food_voice/data/test_a.zip', 'test_a.zip')
    !unzip -qq test_a.zip
    !\rm test_a.zip

INFO:root:Using MoXing-v1.17.3-43fbf97f
INFO:root:Using OBS-Python-SDK-3.20.7


## 环境要求

- TensorFlow的版本：2.0 + 
- keras
- sklearn
- librosa

In [2]:
# 基本库

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import MinMaxScaler

## 加载深度学习框架

In [3]:
# 搭建分类模型所需要的库

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout
from tensorflow.keras.utils import to_categorical 

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

## 加载音频处理库

In [4]:
!pip install librosa

Looking in indexes: http://repo.myhuaweicloud.com/repository/pypi/simple
     |████████████████████████████████| 183 kB 7.7 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 9.1 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 13.7 MB/s eta 0:00:01
     |████████████████████████████████| 323 kB 37.2 MB/s eta 0:00:01
     |████████████████████████████████| 402 kB 43.0 MB/s eta 0:00:01
     |████████████████████████████████| 112 kB 78.7 MB/s eta 0:00:01
  Created wheel for librosa: filename=librosa-0.8.0-py3-none-any.whl size=201375 sha256=9789da56db3c5401915f75c46a63bf937daae551666146e449590c09444105ad
  Stored in directory: /home/ma-user/.cache/pip/wheels/75/11/1b/50c28437b2f3ee88f29ebbb9362696a67edac9b601e24c8d6b
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23141 sha256=49be19a4a9a93933068b3e1df56a36bc6a2702ae1763714f0f7416e7f06b2d15
  Stored in directory: /home/ma-user/.cache/pip/wheels/39/d5/3a/0b55f8d044980666880ac26a1a76

In [5]:
# 其他库

import os
import librosa
import librosa.display
import glob 

INFO:matplotlib.font_manager:Generating new fontManager, this may take some time...


## 特征提取以及数据集的建立

In [6]:
feature = []
label = []
# 建立类别标签，不同类别对应不同的数字。
label_dict = {'aloe': 0, 'burger': 1, 'cabbage': 2,'candied_fruits':3, 'carrots': 4, 'chips':5,
                  'chocolate': 6, 'drinks': 7, 'fries': 8, 'grapes': 9, 'gummies': 10, 'ice-cream':11,
                  'jelly': 12, 'noodles': 13, 'pickles': 14, 'pizza': 15, 'ribs': 16, 'salmon':17,
                  'soup': 18, 'wings': 19}
label_dict_inv = {v:k for k,v in label_dict.items()}

In [7]:
from tqdm import tqdm
def extract_features(parent_dir, sub_dirs, max_file=10, file_ext="*.wav"):
    c = 0
    label, feature = [], []
    for sub_dir in sub_dirs:
        for fn in tqdm(glob.glob(os.path.join(parent_dir, sub_dir, file_ext))[:max_file]): # 遍历数据集的所有文件
            
           # segment_log_specgrams, segment_labels = [], []
            #sound_clip,sr = librosa.load(fn)
            #print(fn)
            label_name = fn.split('/')[-2]
            label.extend([label_dict[label_name]])
            X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
            mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
            feature.extend([mels])
            
    return [feature, label]

In [8]:
# 自己更改目录
parent_dir = './train_sample/'
save_dir = "./"
folds = sub_dirs = np.array(['aloe','burger','cabbage','candied_fruits',
                             'carrots','chips','chocolate','drinks','fries',
                            'grapes','gummies','ice-cream','jelly','noodles','pickles',
                            'pizza','ribs','salmon','soup','wings'])

# 获取特征feature以及类别的label
temp = extract_features(parent_dir,sub_dirs,max_file=100)

100%|██████████| 35/35 [00:03<00:00, 11.07it/s]


In [9]:
temp = np.array(temp)
data = temp.transpose()

/home/ma-user/anaconda3/envs/TensorFlow-2.1.0/lib/python3.7/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  if __name__ == '__main__':


In [10]:
# 获取特征
X = np.vstack(data[:, 0])

# 获取标签
Y = np.array(data[:, 1])
print('X的特征尺寸是：',X.shape)
print('Y的特征尺寸是：',Y.shape)

X的特征尺寸是： (1000, 128)
Y的特征尺寸是： (1000,)


In [11]:
# 在Keras库中：to_categorical就是将类别向量转换为二进制（只有0和1）的矩阵类型表示
Y = to_categorical(Y)

In [12]:
'''最终数据'''
print(X.shape)
print(Y.shape)

(1000, 128)
(1000, 20)


In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify=Y)
print('训练集的大小',len(X_train))
print('测试集的大小',len(X_test))

训练集的大小 750
测试集的大小 250


In [14]:
X_train = X_train.reshape(-1, 16, 8, 1)
X_test = X_test.reshape(-1, 16, 8, 1)

## 建立模型

### 搭建CNN网络

In [15]:
model = Sequential()

# 输入的大小
input_dim = (16, 8, 1)

model.add(Conv2D(64, (3, 3), padding = "same", activation = "tanh", input_shape = input_dim))# 卷积层
model.add(MaxPool2D(pool_size=(2, 2)))# 最大池化
model.add(Conv2D(128, (3, 3), padding = "same", activation = "tanh")) #卷积层
model.add(MaxPool2D(pool_size=(2, 2))) # 最大池化层
model.add(Dropout(0.1))
model.add(Flatten()) # 展开
model.add(Dense(1024, activation = "tanh"))
model.add(Dense(20, activation = "softmax")) # 输出层：20个units输出20个类的概率

# 编译模型，设置损失函数，优化方法以及评价标准
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 8, 64)         640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 4, 64)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 2, 128)         0         
_________________________________________________________________
dropout (Dropout)            (None, 4, 2, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1

In [17]:
# 训练模型
model.fit(X_train, Y_train, epochs = 10, batch_size = 1, validation_data = (X_test, Y_test))

Train on 750 samples, validate on 250 samples
Epoch 1/10
750/750 [==============================] - 6s 8ms/sample - loss: 3.1153 - accuracy: 0.1040 - val_loss: 2.9766 - val_accuracy: 0.1440
Epoch 2/10
750/750 [==============================] - 2s 3ms/sample - loss: 2.8798 - accuracy: 0.1680 - val_loss: 2.8147 - val_accuracy: 0.1680
Epoch 3/10
750/750 [==============================] - 2s 3ms/sample - loss: 2.6715 - accuracy: 0.2160 - val_loss: 3.1164 - val_accuracy: 0.1760
Epoch 4/10
750/750 [==============================] - 2s 3ms/sample - loss: 2.5899 - accuracy: 0.2693 - val_loss: 3.0409 - val_accuracy: 0.1560
Epoch 5/10
750/750 [==============================] - 2s 3ms/sample - loss: 2.5447 - accuracy: 0.2760 - val_loss: 2.9097 - val_accuracy: 0.2040
Epoch 6/10
750/750 [==============================] - 2s 3ms/sample - loss: 2.3393 - accuracy: 0.3173 - val_loss: 2.8968 - val_accuracy: 0.1960
Epoch 7/10
750/750 [==============================] - 2s 3ms/sample - loss: 2.1059 - accur

### 预测测试集

In [18]:
def extract_features(test_dir, file_ext="*.wav"):
    feature = []
    for fn in tqdm(glob.glob(os.path.join(test_dir, file_ext))[:]): # 遍历数据集的所有文件
        X, sample_rate = librosa.load(fn,res_type='kaiser_fast')
        mels = np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0) # 计算梅尔频谱(mel spectrogram),并把它作为特征
        feature.extend([mels])
    return feature

In [19]:
X_test = extract_features('./test_a/')

100%|██████████| 2000/2000 [02:51<00:00, 11.64it/s]


In [20]:
X_test = np.vstack(X_test)
predictions = model.predict(X_test.reshape(-1, 16, 8, 1))

In [21]:
preds = np.argmax(predictions, axis = 1)
preds = [label_dict_inv[x] for x in preds]

path = glob.glob('./test_a/*.wav')
result = pd.DataFrame({'name':path, 'label': preds})

result['name'] = result['name'].apply(lambda x: x.split('/')[-1])
result.to_csv('submit.csv',index=None)

In [22]:
!ls ./test_a/*.wav | wc -l

2000


In [23]:
!wc -l submit.csv

2001 submit.csv
